In [62]:
import sys
sys.path.insert(0, 'datasets/')
sys.path.insert(0, 'models/research/audioset/vggish/')
import dataset
import os
import numpy as np
import torch
import torch.nn as nn
import vggish as vggish_import
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Microphone Locations
mic_height = 50.3125
feet = 12
y_tile = 23.5
x_tile = 11 +7/8

camera_origin_location = np.array([-6*x_tile-5.75, -y_tile, 45+13/16])*25.4

mic_1 = np.array([-11*x_tile - 1, -5*y_tile - 6-3/8, mic_height]) * 25.4 
mic_4= np.array([-11*x_tile + 1.25 + 1/16, 2.5*y_tile+3.75, mic_height]) * 25.4
mic_6 = np.array([5+3/8, 2+1/8, mic_height]) * 25.4
mic_9 = np.array([-3, -6*y_tile - 0.5, mic_height]) * 25.4


mic_xyzs = np.stack((mic_1,mic_4, mic_6, mic_9),axis=0)

SPEAKER_BOTTOM_RIGHT_Y = (1200.15 + 1196.975 + 1206.5)/ 3
SPEAKER_BOTTOM_RIGHT_X = (88.9 + 107.95 + 101.6) / 3
SPEAKER_BOTTOM_LEFT_Y = (1327.15 + 1311.55712764 + 1317.625) / 3
SPEAKER_BOTTOM_LEFT_X = - 76.98583188

speaker_xyz_bottom_right = np.array([SPEAKER_BOTTOM_RIGHT_X, SPEAKER_BOTTOM_RIGHT_Y, 44.5*25.4])
speaker_xyz_bottom_left = np.array([SPEAKER_BOTTOM_LEFT_X, SPEAKER_BOTTOM_LEFT_Y, 44.5*25.4])
speaker_xyz_top_right = np.array([SPEAKER_BOTTOM_RIGHT_X, SPEAKER_BOTTOM_RIGHT_Y, (44.5+17)*25.4])
speaker_xyz_top_left = np.array([SPEAKER_BOTTOM_LEFT_X, SPEAKER_BOTTOM_LEFT_Y, (44.5+17)*25.4])
speaker_xyz = (speaker_xyz_bottom_right+speaker_xyz_bottom_left+speaker_xyz_top_right+speaker_xyz_top_left)/4


walls = None
x_min = - 4000
x_max = 500
y_min = -4000
y_max = 2000

mic_xyzs_base = np.stack((mic_1, mic_4, mic_6, mic_9), axis=0)

In [19]:
import room_config
DATASET_BASE_PATH = "/home/onyxia/work/ia_data_hack_acoustic/data/LivingRoom_preprocessed_hack"

DATASET_PATH = (f"{DATASET_BASE_PATH}/Human1")

#Load dataset attributes - the treated room is called "darkroom" ->> replace by the right values
dr = room_config.Dataset(room_config.RoomSetup(speaker_xyz,
                mic_xyzs_base,
                x_min,
                x_max,
                y_min,
                y_max,
                walls), DATASET_PATH)


print("Living Room Dataset Loaded")

Living Room Dataset Loaded


In [20]:
centroid = np.load(f"{DATASET_PATH}/centroid.npy")
print(f"humains : {centroid.shape}")

humains : (1000, 2)


In [21]:
deconv = np.load(f"{DATASET_PATH}/deconvoled_trim.npy", mmap_mode='r')
print(f"rirs : {deconv.shape}")

rirs : (1000, 4, 667200)


In [22]:
train_indices = np.load("indices/train_indices.npy")
train_xys = centroid[train_indices]

print("Shape of Training xys")
print(train_xys.shape)

Shape of Training xys
(800, 2)


In [25]:
import random
rand = random.Random(10)
    
np.random.seed(0)

In [26]:
mic_indices = [0,1,2,3] #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]#[1, 4, 6, 9] #[8, 0, 5, 3] #mic_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
deconv = deconv[:, mic_indices, :]
print(deconv.shape)

(1000, 4, 667200)


In [28]:
train_indices = np.load("indices/train_indices.npy")
valid_indices = np.load("indices/valid_indices.npy")
test_indices = np.load("indices/test_indices.npy")

In [29]:
train_xy = centroid[train_indices]
valid_xy = centroid[valid_indices]
test_xy = centroid[test_indices]

train_mean = np.mean(train_xy, axis=0)
train_std = np.std(train_xy, axis=0)

print(train_std)
print(np.max(train_xy, axis=0))
print(np.min(train_xy, axis=0))
train_xy = (train_xy - train_mean) / (train_std + 1e-8)
valid_xy = (valid_xy - train_mean) / (train_std + 1e-8)
test_xy =  (test_xy - train_mean) / (train_std + 1e-8)

[1065.49002101 1413.40889991]
[  56.83489407 1542.25968982]
[-4002.15528594 -3499.88767611]


In [33]:
epsilon = 1e-2
norm_val_min = np.min(np.concatenate((train_xy, valid_xy), axis=0))
norm_val_range = np.max(np.concatenate((train_xy, valid_xy), axis=0)) - norm_val_min

def postprocess_net_output(output):
    output[:, :2] = norm_val_range * ((torch.tanh(output[:, :2]) * (1 + epsilon)) + 1) / 2 + norm_val_min
    return output


train_std_cuda = torch.Tensor(train_std)
train_mean_cuda = torch.Tensor(train_mean)

def unnormalize(xy):
    return xy*(train_std_cuda + 1e-8) + train_mean_cuda

def resample(audio, ir=48000, tr=16000):

    resampled_waveform = F.resample(
    audio,
    ir,
    tr,
    lowpass_filter_width=64,
    rolloff=0.9475937167399596,
    resampling_method="kaiser_window",
    beta=14.769656459379492,
    )

In [34]:
train_waves = deconv[train_indices, :]
# 30950 seems to be the rough cutoff after which vggish treats the input as two examples.
valid_waves = deconv[valid_indices, :]
#Test Waves
test_waves = deconv[test_indices, :]
offset = 0
precutoff = 92850

In [35]:
train_waves = train_waves[..., (offset):(offset+precutoff)]
valid_waves = valid_waves[..., (offset):(offset+precutoff)]
test_waves = test_waves[..., (offset):(offset+precutoff)]

train_waves = torch.Tensor(train_waves)
train_xy = torch.Tensor(train_xy)

valid_waves = torch.Tensor(valid_waves)
valid_xy = torch.Tensor(valid_xy)

test_waves = torch.Tensor(test_waves)
test_xy = torch.Tensor(test_xy)

In [36]:
vggish_cutoff = 30950

train_waves = train_waves[..., :vggish_cutoff]
valid_waves = valid_waves[..., :vggish_cutoff]
test_waves = test_waves[..., :vggish_cutoff]

out_channels = 2

print("Done Resampling")

Done Resampling


In [69]:
vggish_model_urls = {
    'vggish': 'https://github.com/harritaylor/torchvggish/'
              'releases/download/v0.1/vggish-10086976.pth',
    'pca': 'https://github.com/harritaylor/torchvggish/'
           'releases/download/v0.1/vggish_pca_params-970ea276.pth'
}

def vggish_test(pretrained=True, **kwargs):
    model = vggish_import.VGGish(urls=vggish_model_urls, pretrained_net=pretrained, postprocess=pretrained, **kwargs)
    return model

In [66]:
class VGGModel(torch.nn.Module):
        
    def __init__(self, fs=16000, pretrained=True):
        super(VGGModel, self).__init__()
        # self.vggish = torch.hub.load('harritaylor/torchvggish', 'vggish')
        self.vggish = vggish_test(pretrained=pretrained)
        self.vggish.eval()
        self.fs = fs
        
    def forward(self, x):
        x = (self.vggish.forward(torch.unsqueeze(x, axis=1), fs=self.fs) / 127) - 1
        # x = (self.vggish.forward(np.expand_dims(x, axis=1), fs=self.fs) / 127) - 1
        return x    

In [67]:
def get_finetune_model(pretrained=True, frozen=True, out_channels=2, dropout=False, in_channels=1):
    # TODO(sclarke): This pretrained model is based on 16kHz samples, and automatically downsamples input audio to 16k
    vggish_model = VGGModel(fs=room_config.PSEUDO_SAMPLE_RATE, pretrained=pretrained)
    if pretrained and frozen:
        for param in vggish_model.parameters():
            param.requires_grad = False
    p_dropout = 0.5 if dropout else 0.0
    finetune_model = torch.nn.Sequential(
        vggish_model,
        torch.nn.Linear(in_features=128, out_features=256, bias=True),
        torch.nn.ReLU(),
        torch.nn.Dropout(p_dropout),
        torch.nn.Linear(in_features=256, out_features=256, bias=True),
        torch.nn.ReLU(),
        torch.nn.Dropout(p_dropout),
        # torch.nn.Linear(in_features=256, out_features=256, bias=True),
        # torch.nn.ReLU(),
        torch.nn.Linear(in_features=256, out_features=out_channels, bias=True)
    )
    
    return finetune_model

In [72]:
train_waves = torch.mean(train_waves)
valid_waves = torch.mean(valid_waves)
test_waves = torch.mean(test_waves)

net = get_finetune_model(pretrained=False, frozen=False, out_channels=out_channels)
#test = VGGModel(fs=room_config.PSEUDO_SAMPLE_RATE, pretrained=True)
print('Using VGGish-based model')

Using VGGish-based model


In [ ]:
total_params = sum(p.numel() for p in net.parameters())
trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)

print('Total parameters: %i'%total_params)
print('Trainable parameters: %i'%trainable_params)

xy_loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(params=net.parameters(), betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.999995)

In [81]:
N_iter = 10
train_losses = []
train_xy_losses = []
valid_losses = []
valid_xy_losses = []
step_count = 0

In [ ]:
for n in range(args.num_epochs):
    print('Reshuffling for Epoch %i'%n, flush=True)
    rand_idx = np.random.permutation(train_waves.shape[0])
    net.train()
    optimizer.zero_grad()
    for i in range(N_iter):
        curr_idx = rand_idx[i*args.batch_size:(i+1)*args.batch_size]
        net_out = net(train_waves[curr_idx, :])
        results = postprocess_net_output(net_out)
        xy_loss = xy_loss_fn(results[:, :2], train_xy[curr_idx, :2])
        loss = xy_loss
        optimizer.zero_grad()
        loss.backward()
        train_loss = loss.item()
        train_losses.append((step_count, train_loss))
        train_xy_losses.append((step_count, xy_loss.item()))
        step_count+=1
        optimizer.step()
        scheduler.step()

    net.eval()
    valid_loss_xy_arr = np.zeros(valid_waves.shape[0], dtype=np.float32)
    valid_loss_arr = np.zeros(valid_waves.shape[0], dtype=np.float32)
    for i in range(valid_waves.shape[0]):
        with torch.no_grad():
            results = torch.squeeze(postprocess_net_output(net(torch.unsqueeze(valid_waves[i, :], axis=0)).view(-1, 1)))
        xy_loss = xy_loss_fn(results[:2], valid_xy[i, :2])
        valid_loss_xy_arr[i] = xy_loss.item()
        loss = xy_loss
        valid_loss_arr[i] = loss.item()
    valid_xy_loss = np.mean(valid_loss_xy_arr)
    valid_loss = np.mean(valid_loss_arr)
    print('Validation XY Loss: %0.3f'%valid_xy_loss)
    print('Validation Loss: %0.3f'%valid_loss)
    valid_losses.append((step_count, valid_loss))
    valid_xy_losses.append((step_count, valid_xy_loss))


    error_path = "../err"
    if not os.path.exists(error_path):
        os.makedirs(args.error_path)

    np.save(os.path.join(error_path, 'train_losses.npy'), np.array(train_losses, dtype=np.float32))
    np.save(os.path.join(error_path, 'valid_losses.npy'), np.array(valid_losses, dtype=np.float32))

    #Iterate through test
    test_errors = np.zeros(test_waves.shape[0], dtype=np.float32)

    for i in range(test_waves.shape[0]):
        with torch.no_grad():
            results = torch.squeeze(postprocess_net_output(net(torch.unsqueeze(test_waves[i, :], axis=0)).view(-1, 1)))
        
        
        test_errors[i] = torch.norm(unnormalize(results[:2]) - unnormalize(test_xy[i, :2])).item()

    print("TEST ERROR")
    print(test_errors)
    
    print("MEAN TEST ERROR",flush=True)
    print(np.mean(test_errors))
    print("MED TEST ERROR")
    print(np.median(test_errors))
    print("STD TEST ERROR")
    print(np.std(test_errors))

    np.save(os.path.join(args.error_path, 'test_errors.npy'), np.array(test_errors, dtype=np.float32))        


    torch.save({
        'epoch': n,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'train_xy_losses': train_xy_losses,
        'valid_losses': valid_losses,
        'valid_xy_losses': valid_xy_losses,
        'train_mean': train_mean,
        'train_std': train_std,
        'norm_val_min':norm_val_min,
        'norm_val_range':norm_val_range,
        'lr': args.lr,
        }, args.save_path)